In [1]:
import paddle as paddle
import paddle.fluid as fluid
import numpy as np
import matplotlib.pyplot as plt
import sys
import cv2
import os
import random
import math
import PIL.Image as Image
from paddle.fluid.initializer import MSRA
from paddle.fluid.param_attr import ParamAttr
from visualdl import LogWriter

In [2]:
def produce_path_label(path):
    train_list = np.load(path)
    return train_list

def produce_test(path):
    train_list = []
    train_data = open(path).readlines()
    for i in train_data:
        items = i.split(',')
        class_index = int(items[0])
        img = np.array(items[1:]).astype('uint8')
        train_list.append([class_index,img])
    return train_list

train_list = produce_path_label('data/FF_train_list.npy')
test_list = produce_path_label('data/FF_test_list.npy')
class_number = len(train_list)
print(class_number)

25


In [3]:
data_shape = [64,64]
class SE_ResNeXt():
    def __init__(self, layers=50):
        self.layers = layers

    def net(self, input, class_dim=1000):
        layers = self.layers
        supported_layers = [50, 101, 152]
        assert layers in supported_layers, \
            "supported layers are {} but input layer is {}".format(supported_layers, layers)
        if layers == 50:
            cardinality = 32
            reduction_ratio = 16
            depth = [3, 4, 6, 3]
            num_filters = [128, 256, 512, 1024]

            conv = self.conv_bn_layer(
                input=input,
                num_filters=64,
                filter_size=7,
                stride=2,
                act='relu',
                name='conv1', )
            conv = fluid.layers.pool2d(
                input=conv,
                pool_size=3,
                pool_stride=2,
                pool_padding=1,
                pool_type='max',
                use_cudnn=False)
        elif layers == 101:
            cardinality = 32
            reduction_ratio = 16
            depth = [3, 4, 23, 3]
            num_filters = [128, 256, 512, 1024]

            conv = self.conv_bn_layer(
                input=input,
                num_filters=64,
                filter_size=7,
                stride=2,
                act='relu',
                name="conv1", )
            conv = fluid.layers.pool2d(
                input=conv,
                pool_size=3,
                pool_stride=2,
                pool_padding=1,
                pool_type='max',
                use_cudnn=False)
        elif layers == 152:
            cardinality = 64
            reduction_ratio = 16
            depth = [3, 8, 36, 3]
            num_filters = [128, 256, 512, 1024]

            conv = self.conv_bn_layer(
                input=input,
                num_filters=64,
                filter_size=3,
                stride=2,
                act='relu',
                name='conv1')
            conv = self.conv_bn_layer(
                input=conv,
                num_filters=64,
                filter_size=3,
                stride=1,
                act='relu',
                name='conv2')
            conv = self.conv_bn_layer(
                input=conv,
                num_filters=128,
                filter_size=3,
                stride=1,
                act='relu',
                name='conv3')
            conv = fluid.layers.pool2d(
                input=conv, pool_size=3, pool_stride=2, pool_padding=1, \
                pool_type='max', use_cudnn=False)
        n = 1 if layers == 50 or layers == 101 else 3
        for block in range(len(depth)):
            n += 1
            for i in range(depth[block]):
                conv = self.bottleneck_block(
                    input=conv,
                    num_filters=num_filters[block],
                    stride=2 if i == 0 and block != 0 else 1,
                    cardinality=cardinality,
                    reduction_ratio=reduction_ratio,
                    name=str(n) + '_' + str(i + 1))

        pool = fluid.layers.pool2d(
            input=conv,
            pool_size=7,
            pool_type='avg',
            global_pooling=True,
            use_cudnn=False)
        drop = fluid.layers.dropout(
            x=pool, dropout_prob=0.5, seed=None)
        stdv = 1.0 / math.sqrt(drop.shape[1] * 1.0)
        out = fluid.layers.fc(
            input=drop,
            size=class_dim,
            act='softmax',
            param_attr=ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv),
                name='fc6_weights'),
            bias_attr=ParamAttr(name='fc6_offset'))
        return out

    def shortcut(self, input, ch_out, stride, name):
        ch_in = input.shape[1]
        if ch_in != ch_out or stride != 1:
            filter_size = 1
            return self.conv_bn_layer(
                input, ch_out, filter_size, stride, name='conv' + name + '_prj')
        else:
            return input

    def bottleneck_block(self,
                         input,
                         num_filters,
                         stride,
                         cardinality,
                         reduction_ratio,
                         name=None):
        conv0 = self.conv_bn_layer(
            input=input,
            num_filters=num_filters,
            filter_size=1,
            act='relu',
            name='conv' + name + '_x1')
        conv1 = self.conv_bn_layer(
            input=conv0,
            num_filters=num_filters,
            filter_size=3,
            stride=stride,
            groups=cardinality,
            act='relu',
            name='conv' + name + '_x2')
        conv2 = self.conv_bn_layer(
            input=conv1,
            num_filters=num_filters * 2,
            filter_size=1,
            act=None,
            name='conv' + name + '_x3')
        scale = self.squeeze_excitation(
            input=conv2,
            num_channels=num_filters * 2,
            reduction_ratio=reduction_ratio,
            name='fc' + name)

        short = self.shortcut(input, num_filters * 2, stride, name=name)

        return fluid.layers.elementwise_add(x=short, y=scale, act='relu')

    def conv_bn_layer(self,
                      input,
                      num_filters,
                      filter_size,
                      stride=1,
                      groups=1,
                      act=None,
                      name=None):
        conv = fluid.layers.conv2d(
            input=input,
            num_filters=num_filters,
            filter_size=filter_size,
            stride=stride,
            padding=(filter_size - 1) // 2,
            groups=groups,
            act=None,
            bias_attr=False,
            param_attr=ParamAttr(name=name + '_weights'), )
        bn_name = name + "_bn"
        return fluid.layers.batch_norm(
            input=conv,
            act=act,
            param_attr=ParamAttr(name=bn_name + '_scale'),
            bias_attr=ParamAttr(bn_name + '_offset'),
            moving_mean_name=bn_name + '_mean',
            moving_variance_name=bn_name + '_variance')

    def squeeze_excitation(self,
                           input,
                           num_channels,
                           reduction_ratio,
                           name=None):
        pool = fluid.layers.pool2d(
            input=input,
            pool_size=0,
            pool_type='avg',
            global_pooling=True,
            use_cudnn=False)
        stdv = 1.0 / math.sqrt(pool.shape[1] * 1.0)
        squeeze = fluid.layers.fc(
            input=pool,
            size=num_channels // reduction_ratio,
            act='relu',
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv),
                name=name + '_sqz_weights'),
            bias_attr=ParamAttr(name=name + '_sqz_offset'))
        stdv = 1.0 / math.sqrt(squeeze.shape[1] * 1.0)
        excitation = fluid.layers.fc(
            input=squeeze,
            size=num_channels,
            act='sigmoid',
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv),
                name=name + '_exc_weights'),
            bias_attr=ParamAttr(name=name + '_exc_offset'))
        scale = fluid.layers.elementwise_mul(x=input, y=excitation, axis=0)
        return scale


def SE_ResNeXt50_32x4d():
    model = SE_ResNeXt(layers=50)
    return model

In [4]:
yuzhi = 1
def addGaussianNoise(image): 
    G_Noiseimg = np.array(image)
    G_NoiseNum=random.randint(1,5)
    for i in range(G_NoiseNum): 
        temp_x = np.random.randint(0,image.shape[0])
        temp_y = np.random.randint(0,image.shape[0])
        temp_x_size = np.random.randint(1,5)
        temp_y_size = np.random.randint(1,5)
        temp_x_end = min(image.shape[0],temp_x+temp_x_size)
        temp_y_end = min(image.shape[0],temp_y+temp_y_size)
        for x in range(temp_x,temp_x_end):
            for y in range(temp_y,temp_y_end):
                G_Noiseimg[x][y] = 255
    return G_Noiseimg
def for_iterater_reader(t_list):
    def reader():
        for i in range(0,4000):
            for label in range(0,class_number):
              #  try:
                    #img = cv2.resize(img,(data_shape[0],data_shape[1]))
                    #img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
                    #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                    tmp_ran = train_list[label]
                    ran_int = random.randint(0,len(tmp_ran)-1)#取随机的一个数
                    img = np.array(tmp_ran[ran_int]).reshape(64,64)
                    r_sofang = random.randint(6,12)/10
                    shape = int(r_sofang*data_shape[0])
                    img = np.array(img)
                    _, img = cv2.threshold(img, 10, 255,cv2.THRESH_BINARY)
                    h,w = np.array(img).shape
                    r_x = random.randint(-5,5)
                    r_y = random.randint(-4,4)
                    r_rota = random.randint(-5,5)
                    r_yh = random.randint(1,3)
                    img = Image.fromarray(img)
                    img = img.rotate(r_rota)
                    tmpimg = Image.new('L',(data_shape[0],data_shape[0]))
                    tmpimg.paste(img,(r_x,r_y))
                    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (r_yh, r_yh))
                    tmpimg = cv2.dilate(np.array(tmpimg), element1, iterations = 1)
                    tmpimg = addGaussianNoise(tmpimg)
                    #img = cv2.blur(img,(20,20))#羽化
                    #ret, binary = cv2.threshold(img, 50, 255,cv2.THRESH_BINARY)
                    #element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (10,10))#拓展
                    #img = cv2.erode(binary, element1, iterations = 1)
                    img = np.array(tmpimg)
                    ######################################################
                    img = img.reshape(1,data_shape[0],data_shape[1]).astype('float32')
                    img = (img)/255 
                    yield label,img
            #    except Exception:
             #       print(label)
    return reader

def for_test_reader(t_list):
    def reader():
        for class_index in t_list:
            image = np.array(class_index[1]).reshape(64,64)
            image = Image.fromarray(image).resize((data_shape[0],data_shape[0]))
            image = np.array(image).reshape(1,data_shape[0],data_shape[0]).astype('float32')
            image = (image) / 255.0
            yield class_index[0],image
    return reader

In [5]:
#1.设置地方
place = fluid.CUDAPlace(0)
#2.设置数据和设置标签
label = fluid.layers.data(name = 'label',shape=[1],dtype='int64')
image = fluid.layers.data(name = 'image',shape=[1,data_shape[0],data_shape[1]],dtype='float32')
#3.设置网络和Feeder
feeder = fluid.DataFeeder(place = place , feed_list = [label,image])
net = SE_ResNeXt50_32x4d().net(image,class_number)
#4.设置损失函数和正确率
cost = fluid.layers.cross_entropy(input = net , label = label)
avg_cost = fluid.layers.mean(cost)
acc = fluid.layers.accuracy(input = net , label =label)
#.定义测试程序
test_program = fluid.default_main_program().clone(for_test=True)
#5.设置优化
LR = 0.00001
piecewise_decay = fluid.layers.piecewise_decay([1000,5000,10000], [LR, LR * 0.01,LR * 0.001,LR * 0.0001])
optimizer = fluid.optimizer.AdamOptimizer(learning_rate=piecewise_decay)
optimizer.minimize(avg_cost)
#6.定义Executor
exe = fluid.Executor(place = place)
exe.run(fluid.default_startup_program())

[]

In [6]:
import shutil
import os
def save_inference(pass_id):
     # 保存预测模型
    save_path = 'models/infer_model/'+str(pass_id)+"/"
    # 删除旧的模型文件
    shutil.rmtree(save_path, ignore_errors=True)
    # 创建保持模型文件目录
    os.makedirs(save_path)
    # 保存预测模型
    fluid.io.save_inference_model(save_path, feeded_var_names=[image.name], target_vars=[net], executor=exe)

def save_model(pass_id):
    # 保存参数模型
    save_path = 'models/params_model/'+str(pass_id)+"/"
    # 删除旧的模型文件
    shutil.rmtree(save_path, ignore_errors=True)
    # 创建保持模型文件目录
    os.makedirs(save_path)
    # 保存参数模型
    fluid.io.save_params(executor=exe, dirname=save_path)

def load_model(pass_id):
    # 加载之前训练过的参数模型
    save_path = 'models/params_model/'+str(pass_id)
    if os.path.exists(save_path):
        print('使用参数模型作为预训练模型')
        fluid.io.load_params(executor=exe, dirname=save_path)

In [7]:
#8.数据分批
train_Reader = paddle.batch(reader=paddle.reader.shuffle(for_iterater_reader(train_list),buf_size = 128*128),batch_size=64)
test_Reader = paddle.batch(reader=paddle.reader.shuffle(for_iterater_reader(test_list),buf_size = 128*128),batch_size=64)

In [8]:
print("加载完毕")

加载完毕


In [9]:
#9.训练
step = 0
best_model_precent = 0
error_cout = 20
save_model_name = 5

In [10]:
load_model(save_model_name)

使用参数模型作为预训练模型


In [11]:
 def train():
    for pass_id in range(20000):
        sum_cost=0
        sum_acc=0
        global best_model_precent
        for batch_id , data in enumerate(train_Reader()):
            train_cost, train_acc = exe.run(program = fluid.default_main_program(),
                                           feed = feeder.feed(data),
                                            fetch_list = [avg_cost,acc]
                                           )
            sum_cost = sum_cost + train_cost[0]
            sum_acc = sum_acc + train_acc[0]
            if batch_id % 50 == 0 and batch_id != 0:
                print('Pass：%d, Batch：%d, Cost：%f, Accuracy：%f' % (pass_id , batch_id , sum_cost/51, sum_acc/51))
                sum_cost = 0
                sum_acc = 0
        test_sum_cost=0
        test_sum_acc=0
        test_cout = 0
        for batch_id , data in enumerate(test_Reader()):
            train_cost, train_acc = exe.run(program = test_program,
                                           feed = feeder.feed(data),
                                            fetch_list = [avg_cost,acc]
                                           )
            test_sum_cost = test_sum_cost + train_cost[0]
            test_sum_acc = test_sum_acc + train_acc[0]
            test_cout+=1
        print("test - Pass:",pass_id,"Cost:",test_sum_cost/test_cout,"Acc:",test_sum_acc/test_cout)
        total_model_precent = (test_sum_acc/test_cout)
        print("new BestPrecent:",best_model_precent,"thie Precent:",total_model_precent)
        if total_model_precent > best_model_precent:
            best_model_precent = total_model_precent
            save_model(save_model_name)
            print("Save...model...Best_precent:",best_model_precent)
        print()
train()

Pass：0, Batch：50, Cost：0.054586, Accuracy：0.983762
Pass：0, Batch：100, Cost：0.051817, Accuracy：0.965686
Pass：0, Batch：150, Cost：0.037611, Accuracy：0.972120
Pass：0, Batch：200, Cost：0.030319, Accuracy：0.970282
Pass：0, Batch：250, Cost：0.029016, Accuracy：0.971814
Pass：0, Batch：300, Cost：0.023298, Accuracy：0.973958
Pass：0, Batch：350, Cost：0.026432, Accuracy：0.970895
Pass：0, Batch：400, Cost：0.025929, Accuracy：0.971507
Pass：0, Batch：450, Cost：0.030688, Accuracy：0.972120
Pass：0, Batch：500, Cost：0.031172, Accuracy：0.969975
Pass：0, Batch：550, Cost：0.023604, Accuracy：0.973039
Pass：0, Batch：600, Cost：0.021164, Accuracy：0.973346
Pass：0, Batch：650, Cost：0.022848, Accuracy：0.973958
Pass：0, Batch：700, Cost：0.019420, Accuracy：0.974265
Pass：0, Batch：750, Cost：0.030612, Accuracy：0.970282
Pass：0, Batch：800, Cost：0.026415, Accuracy：0.973039
Pass：0, Batch：850, Cost：0.021064, Accuracy：0.973652
Pass：0, Batch：900, Cost：0.017264, Accuracy：0.974877
Pass：0, Batch：950, Cost：0.027915, Accuracy：0.971814
Pass：0, Batch

KeyboardInterrupt: 

In [ ]:
save_model(save_model_name)

In [ ]:
load_model(save_model_name)

In [12]:
save_inference(save_model_name)

In [ ]:
test_image = test_list[19][20]
plt.figure()
plt.imshow(test_image)
test_image = np.array(test_image).astype('float32').reshape(1,1,64,64)/255
result = exe.run(program = test_program,feed = {'image':test_image,'label':np.array([[1]]).astype('int64')},fetch_list = [net])
print(np.argmax(result))